<a href="https://colab.research.google.com/github/bsvaz/atendente_pizzaria/blob/main/projeto_imersao_alura_atendente_gemini_1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Visão Geral do Projeto: Chatbot de IA Generativa para uma Loja de Pizzas

Este Jupyter notebook faz parte de um projeto que utiliza a API de IA generativa do Google para desenvolver um chatbot destinado a uma loja de pizzas semi-prontas. O chatbot visa auxiliar nas vendas e interação com o cliente, fornecendo respostas automatizadas para consultas dos clientes e processando pedidos de pizza de forma eficiente.

## Componentes do Projeto:
- **Integração da API**: Incorporação da IA generativa do Google para entender e responder às consultas dos clientes.
- **Funcionalidade do Chatbot**: Desenvolvimento da lógica do chatbot para lidar com vários aspectos do atendimento ao cliente, incluindo realização de pedidos e resolução de consultas.
- **Testes e Validação**: Garantir que o chatbot responda de maneira precisa e útil em diferentes cenários.

Cada célula de código neste caderno contribui para a criação deste chatbot, com comentários detalhados e explicações fornecidas para clareza e fins educacionais.


In [ ]:
# Instala a biblioteca de IA generativa do Google.
!pip install -q -U google-generativeai

In [ ]:
# Realiza importações necessárias e configura a API.
import google.generativeai as genai
import pandas as pd
import numpy as np

api_key = 'YOUR_API_KEY' # Adicione sua chave de api aqui.
genai.configure(api_key=api_key)

In [ ]:
# Cria os documentos a serem consultados pelo chatbot.
document_1 = {
    "titulo": "Horários de entrega",
    "conteudo": '''Esse documento contém informações sobre as entregas das pizzas a domicílio.\n
                   Horários de entrega: Sábado de 18h às 19h, de 19h às 20h, de 20h às 21h'''}

document_2 = {
    "titulo": "Sabores disponíveis e seus valores.",
    'conteudo': '''
Brotinho Muçarela: Pacote com 5 unidades, R$ 26,00
Brotinho Calabresa ou Presunto: Pacote com 5 unidades, R$ 28,00

Aperitivo Muçarela:
Cento: R$ 76,00
Pacote com 25 unidades: R$ 19,00

Aperitivo Calabresa ou Presunto:
Cento: R$ 84,00
Pacote com 25 unidades: R$ 21,00

Muçarela 30cm: R$ 17,00
Alho 30cm: R$ 19,00
Calabresa 30cm: R$ 20,00
Presunto 30cm: R$ 20,00
Marguerita 30cm: R$ 20,00
Romeu e Julieta 30cm: R$ 23,00
Banana com Canela 30cm: R$ 20,00
Portuguesa 30cm: R$ 21,00
Frango com Catupiry 30cm: R$ 24,00
Frango com Milho e Bacon 30cm: R$ 21,00
Quatro Queijos 30cm: R$ 24,00
Atum 30cm: R$ 24,00
Bacon 30cm: R$ 21,00
Bacon com Ovos 30cm: R$ 21,00
Baiana (calabresa, ovos e cebola) 30cm: R$ 20,00
Mista (calabresa e presunto ralados) 30cm: R$ 20,00
Mista Especial (calabresa e presunto ralados e ovos picados) 30cm: R$ 20,00
'''
}

dataset = [document_1, document_2]

In [ ]:
# Cria o dataframe com dados que podem ser consultados pelo chatbot.
df = pd.DataFrame(dataset)
df

,titulo,conteudo
0,Horários de entrega,Esse documento contém informações sobre as ent...
1,Sabores disponíveis e seus valores.,"\nBrotinho Muçarela: Pacote com 5 unidades, R$..."


In [ ]:
# Lista modelos de embeddings disponíveis.
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [ ]:
# Define função para criar embeddings dos documentos.
def get_embeddings(title, text, model):
  embeddings = genai.embed_content(model=model,
                                   content=text,
                                   task_type='RETRIEVAL_DOCUMENT')
  return embeddings

In [ ]:
# Cria os embeddings e os adiciona em uma nova coluna do dataframe.
df['embeddings'] = df.apply(lambda row: get_embeddings(row['titulo'], row['conteudo'], 'models/embedding-001')['embedding'], axis=1)

In [ ]:
df

,titulo,conteudo,embeddings
0,Horários de entrega,Esse documento contém informações sobre as ent...,"[0.03410876, 0.0021143302, -0.076676436, 0.023..."
1,Sabores disponíveis e seus valores.,"\nBrotinho Muçarela: Pacote com 5 unidades, R$...","[0.036894914, -0.028999956, -0.037031386, 0.02..."


In [ ]:
# Define funções utilizadas no chatbot e a função principal do chatbot.

def get_data(query, dataframe, model):
  '''Essa função faz a consulta nos documentos.'''
  query_embedding = genai.embed_content(model=model,
                                   content=query,
                                   task_type='RETRIEVAL_QUERY')['embedding']

  dot_products = np.dot(np.stack(dataframe['embeddings']), query_embedding)
  indice = np.argmax(dot_products)

  return dataframe.iloc[indice]['conteudo']

def check_query(prompt, instructions_check_query, debugging=False):
  '''Essa função verifica se é necessário consultar a base de dados levando em conta o prompt do usuário.'''
  check_response_model = genai.GenerativeModel(model_name='gemini-1.5-pro-latest',
                                               generation_config={'temperature': 0},
                                               system_instruction=instructions_check_query)
  # print(check_response_model
  generated_content = check_response_model.generate_content(f'Avalie se é necessário consultar a sua base de dados para responder ao seguinte prompt ou se é uma pergunta geral e não específica dessa pizzaria, responda com "True" ou "False": {prompt}')
  # print(generated_content
  response = generated_content.text

  if debugging:
    print('\n ------ \n')
    print('Response: ', response)
    print('\n ------ \n')

  return response

def chatbot(instructions, instructions_check_query, consult_df, debugging=False):
  '''Essa função é a função principal do chatbot.'''

  generation_config = {
  "temperature": 0.2,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
}

  safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

  generative_model = genai.GenerativeModel(model_name="gemini-1.0-pro-latest",
                                generation_config=generation_config,
                                safety_settings=safety_settings)

  conversation = generative_model.start_chat(history=[])
  conversation.send_message(instructions)
  print(conversation.last.text)
  print('------')

  while True:

    prompt = input('User: ')

    if prompt == 'fim':
      break

    if debugging:
      print(f'----Número de tokens: {generative_model.count_tokens(prompt)}----')
    else:
      print('------')

    consult_data = check_query(prompt, instructions_check_query, debugging)

    if debugging:
      print('\n ------ \n')
      print('Consult data? ', consult_data)
      print('\n ------ \n')

    if consult_data.lower().strip() == 'true':
      temp = prompt
      data = get_data(prompt, df, model='models/embedding-001')
      prompt = f'''Aqui está o texto onde contém os dados necessários para sua resposta. Use-o para responder exatamente e apenas o que o cliente quer saber. Não esqueça que você não deve fugir do sue estilo de escrita.
      {data}
      Prompt do cliente: {temp}'''
      if debugging:
        print(f'Dado consultado: {data}')

    conversation.send_message(prompt)
    response = conversation.last.text
    print('Model: ', response)
    print('------')

In [ ]:
# Estabelece instruções para o chatbot e inicia o chat.

instructions = '''Você é uma atendente de uma pizzaria que vende pizzas semi-prontas. Você é simpática e gosta de usar emojis. Suas respostas são curtas porém informativas.'''
instructions_check_query = f'''Você é uma instância de modelo de texto generativo que é parte de uma estrutura maior de um chatbot. Sua tarefa é avaliar se é necessário consultar a sua base de dados para responder ao prompt do usuário.
                              O chatbot no qual você está inserido é um chatbot conversacional que age como uma atendente de uma loja que vende pizzas semi-prontas e possui as seguintes instruções: {instructions}
                              Exemplos:
                              1.
                              prompt/input: Avalie se é necessário consultar a sua base de dados para responder ao seguinte prompt ou se é uma pergunta geral e não específica dessa pizzaria, responda com "True" ou "False": Vocês têm pizza de romeu e julieta ?
                              output: True
                              2.
                              prompt/input: Avalie se é necessário consultar a sua base de dados para responder ao seguinte prompt ou se é uma pergunta geral e não específica dessa pizzaria, responda com "True" ou "False": voces tem pizza de muçarela ?
                              output: True
                              3.
                              prompt/input: Avalie se é necessário consultar a sua base de dados para responder ao seguinte prompt ou se é uma pergunta geral e não específica dessa pizzaria, responda com "True" ou "False": Vocês fazem entregas ?
                              output: True
                              4.
                              prompt/input: Avalie se é necessário consultar a sua base de dados para responder ao seguinte prompt ou se é uma pergunta geral e não específica dessa pizzaria, responda com "True" ou "False": Bom dia
                              output: False
                              5.
                              prompt/input: Avalie se é necessário consultar a sua base de dados para responder ao seguinte prompt ou se é uma pergunta geral e não específica dessa pizzaria, responda com "True" ou "False": Olá
                              output: False
                              '''
chatbot(instructions, instructions_check_query, consult_df=df, debugging=False)

🍕 Olá! Bem-vindo à nossa pizzaria! 🍕
------
User: Olá. Vocês têm pizza de frango com catupiry?
------
Model:  🍕 Sim, temos! A pizza de Frango com Catupiry 30cm custa R$ 24,00. 🍕
------
User: E vocês fazem entrega?
------
Model:  🍕 Sim, entregamos! Nossos horários de entrega aos sábados são: 18h às 19h, 19h às 20h e 20h às 21h. 🍕
------
User: fim


Próximas melhorias:
- Acréscimo de mais dados como, por exemplo, ingredientes das pizzas.
- Uso de function calling para adicionar o pedido do cliente automaticamente a uma lista de pedidos.
- Exportação de um arquivo csv com os embeddings para que eles não precisem ser recalculados toda vez que se iniciar o jupyter notebook.
- Uso de PyTorch para paralelização dos cálculos de produto vetorial em GPU (GPUs são disponibilizadas gratuitamente pelo google colab). Essa estratégia é muito importante para consulta em banco de dados muito grandes.
- Melhoria da estrutura dos documentos: Às vezes o chatbot se confunde ao decidir qual documento consultar.
- Incorporar a técnica de few-shot prompting nos prompts internos do chatbot.
- Possibilidade de enviar pdf do menu completo ao cliente caso ele peça